In [1]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.svm import SVC

from bayes_opt import BayesianOptimization
from bayes_opt.util import Colours
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn import metrics
from sklearn.svm import SVC
import xgboost
from xgboost import XGBClassifier
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
from sklearn.ensemble import RandomForestClassifier

c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
c:\users\hp\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The skle

In [2]:
# load dataset
dataset = pd.read_csv(r"C:\Users\HP\Desktop\Final_COP\TrainN.csv") 
dataset = dataset.drop('Unnamed: 0', axis = 1)

In [3]:
def get_data():
        data = dataset.drop(['PHQ'],axis=1).values #features = X arrays 
        targets = dataset['PHQ'].values #labels = y arrays
        return data, targets

In [4]:
def rfc_cv(n_estimators, min_samples_split, max_features, data, targets):
    """Random Forest cross validation.
    This function will instantiate a random forest classifier with parameters
    n_estimators, min_samples_split, and max_features. Combined with data and
    targets this will in turn be used to perform cross validation. The result
    of cross validation is returned.
    Our goal is to find combinations of n_estimators, min_samples_split, and
    max_features that maximize the AUC
    """
    estimator = RFC(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        max_features=max_features,
        random_state=0,
        class_weight = 'balanced_subsample'
    )
    cval = cross_val_score(estimator, data, targets,
                           scoring='roc_auc', cv=10)
    return cval.mean()



def optimize_rfc(data, targets):
    """Apply Bayesian Optimization to Random Forest parameters."""
    def rfc_crossval(n_estimators, min_samples_split, max_features):
        """Wrapper of RandomForest cross validation.
        Notice how we ensure n_estimators and min_samples_split are casted
        to integer before we pass them along. Moreover, to avoid max_features
        taking values outside the (0, 1) range, we also ensure it is capped
        accordingly.
        """
        return rfc_cv(
            n_estimators=int(n_estimators),
            min_samples_split=int(min_samples_split),
            max_features=max(min(max_features, 0.999), 1e-3),
            data=data,
            targets=targets
        )

    optimizer = BayesianOptimization(
        f=rfc_crossval,
        pbounds={
            "n_estimators": (1, 2000),
            "min_samples_split": (2, 55),
            "max_features": (0.1, 0.999)
         },
        random_state=0,
        verbose=2
    )
    optimizer.maximize(n_iter=5)

    print("Final result:", optimizer.max)   

In [5]:
data, targets = get_data()
optimize_rfc(data, targets)

|   iter    |  target   | max_fe... | min_sa... | n_esti... |
-------------------------------------------------------------
|  1        |  0.6769   |  0.5934   |  39.91    |  1.206e+0 |
|  2        |  0.6742   |  0.5898   |  24.45    |  1.292e+0 |
|  3        |  0.6815   |  0.4934   |  49.26    |  1.927e+0 |
|  4        |  0.6785   |  0.4447   |  43.96    |  1.058e+0 |
|  5        |  0.6693   |  0.6107   |  51.06    |  143.0    |
|  6        |  0.6806   |  0.4624   |  54.86    |  1.996e+0 |
|  7        |  0.6785   |  0.7439   |  54.98    |  1.605e+0 |
|  8        |  0.6752   |  0.3222   |  2.455    |  2e+03    |
|  9        |  0.6792   |  0.6645   |  54.85    |  1.993e+0 |
|  10       |  0.6732   |  0.9948   |  54.95    |  1.992e+0 |
Final result: {'target': 0.6815454027454028, 'params': {'max_features': 0.49339090292516063, 'min_samples_split': 49.26396904145023, 'n_estimators': 1927.3618582415575}}
